# OpenAPI Currency Exchange Tutorial

💱 **Learn how to use OpenAPI tools with Azure AI Agents for currency exchange!**

This tutorial demonstrates:
1. **Part 1: Azure AI Foundry SDK** - Using OpenAPI tools for currency exchange
2. **Part 2: Semantic Kernel SDK** - Azure AI Agents with OpenAPI tools

**OpenAPI tools** allow agents to call external APIs defined by OpenAPI specifications.

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name

**For Semantic Kernel:**
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI endpoint
- `AZURE_OPENAI_API_KEY`: Your Azure OpenAI API key
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`: Your chat model deployment

![Azure Tools](images/azure_tools.gif)

In [1]:
# Install required packages
# !pip install azure-ai-agents azure-identity semantic-kernel

import os
import json
import json
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import OpenApiTool, OpenApiAnonymousAuthDetails, MessageRole
from azure.identity import DefaultAzureCredential

print("✅ Packages imported successfully!")

✅ Packages imported successfully!


---

# Part 1: Azure AI Foundry SDK - OpenAPI Currency Exchange

The Foundry SDK provides **OpenApiTool** to call external APIs from agent responses.

**Architecture:**
```
Agent → OpenAPI Tool → Frankfurter API (currency exchange)
```

In [2]:
# Load the currency exchange OpenAPI specification
currency_spec_path = os.path.join("openapi_files", "currency_exchange.json")

with open(currency_spec_path, "r") as f:
    currency_openapi_spec = json.loads(f.read())

print("📊 Currency exchange OpenAPI spec loaded")
print(f"API Title: {currency_openapi_spec['info']['title']}")
print(f"API Version: {currency_openapi_spec['info']['version']}")

📊 Currency exchange OpenAPI spec loaded
API Title: Frankfurter Exchange Rates API
API Version: 1.0


In [3]:
# Create Azure AI Agents client
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

print("🔗 Connected to Azure AI Agents service")

🔗 Connected to Azure AI Agents service


In [4]:
# Create OpenAPI tool for currency exchange
auth = OpenApiAnonymousAuthDetails()

currency_tool = OpenApiTool(
    name="currency_exchange",
    spec=currency_openapi_spec,
    description="Get the latest foreign exchange rates from Frankfurter API",
    auth=auth
)

print("💱 Currency exchange OpenAPI tool created")
print(f"Available operations: {len(currency_tool.definitions)}")

💱 Currency exchange OpenAPI tool created
Available operations: 1


In [5]:
# Create agent with OpenAPI tool
currency_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="currency_exchange_agent",
    instructions="You are a helpful currency exchange agent. Use the currency exchange API to get latest exchange rates. Always provide clear explanations of the rates.",
    tools=currency_tool.definitions
)

print(f"💰 Currency exchange agent created: {currency_agent.id}")

💰 Currency exchange agent created: asst_Pvq8NHv81JzZ7NH8DuTZw41m


In [6]:
# Test the currency exchange agent
# Create conversation thread
thread = agents_client.threads.create()
print(f"💬 Thread created: {thread.id}")

# Ask about exchange rates
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="What is the current USD to EUR exchange rate? Also show me USD to GBP and USD to JPY."
)

# Process the request
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=currency_agent.id
)

print(f"✅ Run status: {run.status}")

if run.status == "failed":
    print(f"❌ Run failed: {run.last_error}")

# Display the response
messages = agents_client.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant" and msg.text_messages:
        print(f"\n💱 Currency Agent Response:")
        print(msg.text_messages[-1].text.value)
        break

💬 Thread created: thread_G5uTmu0vwRkzfOIR72snM2RW
✅ Run status: RunStatus.COMPLETED

💱 Currency Agent Response:
Here are the latest exchange rates with USD as the base currency:

- 1 USD = 0.85507 EUR (Euro)
- 1 USD = 0.7298 GBP (British Pound)
- 1 USD = 144.44 JPY (Japanese Yen)

This means for every 1 US Dollar, you get approximately 0.855 Euros, 0.73 British Pounds, or 144.44 Japanese Yen. Let me know if you need rates for other currencies!


In [7]:
# Test with different base currency
# Create new message in same thread
message2 = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="Now show me EUR to USD, EUR to GBP, and EUR to CHF exchange rates."
)

# Process the request
run2 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=currency_agent.id
)

print(f"✅ Second run status: {run2.status}")

# Display the latest response
messages = agents_client.messages.list(thread_id=thread.id)
print(f"\n💱 Currency Agent Response (EUR base):")
for msg in messages:
    if msg.role == "assistant" and msg.text_messages:
        print(msg.text_messages[-1].text.value)
        break

✅ Second run status: RunStatus.COMPLETED

💱 Currency Agent Response (EUR base):
Here are the latest exchange rates with EUR (Euro) as the base currency:

- 1 EUR = 1.1695 USD (US Dollar)
- 1 EUR = 0.8535 GBP (British Pound)
- 1 EUR = 0.9379 CHF (Swiss Franc)

This means for every 1 Euro, you get about 1.17 US Dollars, 0.85 British Pounds, or 0.94 Swiss Francs. If you need rates for other currencies, feel free to ask!


---

# Part 2: Semantic Kernel SDK - Azure AI Agent with OpenAPI Tools

Semantic Kernel provides **AzureAIAgent** to wrap Azure AI agents with OpenAPI tools.

**Architecture:**
```
Semantic Kernel
    ↓
AzureAIAgent (wraps Azure AI agent with OpenAPI tools)
    ↓
Frankfurter API (currency exchange)
```

In [9]:
# Semantic Kernel imports
import asyncio
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents import AuthorRole

print("📚 Semantic Kernel imports loaded")

📚 Semantic Kernel imports loaded


In [10]:
# Create Azure AI Agent with OpenAPI tools using Semantic Kernel
async def create_sk_currency_agent():
    endpoint = os.environ["PROJECT_ENDPOINT"]
    model_deployment = os.environ["MODEL_DEPLOYMENT_NAME"]

    async with (
        AsyncDefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds, endpoint=endpoint) as sk_client,
    ):
        print("🔗 Creating SK Azure AI Agent with OpenAPI tools...")
        
        # Create Azure AI agent with OpenAPI tool using SK
        agent_definition = await sk_client.agents.create_agent(
            model=model_deployment,
            name="sk_currency_agent",
            instructions="You are an expert currency exchange assistant. Use the currency exchange API to provide accurate, up-to-date exchange rates. Always explain the rates clearly and provide context.",
            tools=currency_tool.definitions
        )
        
        # Wrap as AzureAIAgent
        sk_currency_agent = AzureAIAgent(
            client=sk_client,
            definition=agent_definition
        )
        
        print(f"💰 SK Currency agent created: {agent_definition.id}")
        
        # Test queries
        test_queries = [
            "What's the current exchange rate from USD to THB (Thai Baht)? I'm planning a trip to Thailand.",
            "Show me how much 1000 USD would be worth in EUR, GBP, and CAD today."
        ]
        
        thread = None
        
        try:
            for i, query in enumerate(test_queries, 1):
                print(f"\n🧪 Test Query {i}:")
                print(f"👤 User: {query}")
                
                # Invoke the agent
                async for response in sk_currency_agent.invoke(messages=query, thread=thread):
                    if response.role != AuthorRole.TOOL:
                        print(f"💱 SK Currency Agent: {response.content}")
                    thread = response.thread
        
        finally:
            # Cleanup
            if thread:
                await sk_client.agents.threads.delete(thread.id)
            await sk_client.agents.delete_agent(agent_definition.id)
            print("\n🧹 SK agent cleaned up")

# Run the SK currency agent
await create_sk_currency_agent()
print("\n✅ Semantic Kernel Azure AI Agent with OpenAPI complete!")

🔗 Creating SK Azure AI Agent with OpenAPI tools...
💰 SK Currency agent created: asst_4eipQqUlbJErAcErI1ZDrDgX

🧪 Test Query 1:
👤 User: What's the current exchange rate from USD to THB (Thai Baht)? I'm planning a trip to Thailand.
💱 SK Currency Agent: The current exchange rate from USD to THB (Thai Baht) is approximately 1 USD = 32.51 THB as of today.

This means that for every US dollar you exchange, you will receive about 32.51 Thai Baht. It's a good idea to check if your bank or currency exchange service charges any fees or offers slightly different rates, as the market rate can differ from what you receive.

If you need a calculation for a specific amount or more travel tips, let me know! Safe travels to Thailand!

🧪 Test Query 2:
👤 User: Show me how much 1000 USD would be worth in EUR, GBP, and CAD today.
💱 SK Currency Agent: Based on the latest exchange rates, here’s how much 1,000 USD would be worth in each currency:

- EUR (Euro): 1 USD = 0.85507 EUR
  - 1,000 USD × 0.85507 = 85

In [ ]:
# Final cleanup - delete the Foundry SDK agent
agents_client.delete_agent(currency_agent.id)
print("🧹 All agents cleaned up")

---

## 🔄 Comparison: Foundry SDK vs Semantic Kernel OpenAPI

| Aspect | Foundry SDK | Semantic Kernel (AzureAI) |
|--------|-------------|---------------------------|
| **Setup** | Direct OpenApiTool creation | Azure AI agent with OpenAPI tools |
| **Agent Type** | Native Azure AI agent | AzureAIAgent wrapper |
| **API Calls** | Built-in tool execution | SK orchestration + Azure AI power |
| **Response** | Direct agent response | SK streaming with tool filtering |
| **Context** | Thread-based conversation | SK thread management |

### When to Use Each:

**⚙️ Foundry SDK:**
- Direct Azure AI agent usage
- Simple OpenAPI tool integration
- Native Azure AI features

**🔌 Semantic Kernel:**
- Rich SK orchestration features
- Complex agent workflows
- Leverage existing Azure AI agents in SK pipelines

---

## 💱 OpenAPI Currency Exchange Features

### 🌍 Frankfurter API Capabilities:
- **Real-time rates**: Latest foreign exchange rates
- **Multiple currencies**: 30+ supported currencies
- **Base currency**: Configure any currency as base
- **Free access**: No API key required

### 🔧 OpenAPI Tool Benefits:
- **Auto-discovery**: Agents understand API capabilities
- **Type safety**: OpenAPI schema validation
- **Documentation**: Built-in API documentation
- **Flexibility**: Easy to add new APIs

### 💡 Best Practices:
✅ **Always validate** OpenAPI specifications  
✅ **Handle errors** gracefully in agent instructions  
✅ **Use descriptive** tool names and descriptions  
✅ **Test thoroughly** with different currencies  
✅ **Consider rate limits** for production usage  

---

## 🎯 Key Takeaways

✅ **OpenAPI Tools (Both SDKs):**
- Enable agents to call external APIs
- Use OpenAPI specifications for auto-discovery
- Support various authentication methods

✅ **Foundry SDK:**
- Use `OpenApiTool` for direct API integration
- Perfect for straightforward API calls
- Built-in Azure AI agent capabilities

✅ **Semantic Kernel:**
- Use `AzureAIAgent` for enhanced orchestration
- Leverage SK's streaming and filtering
- Combine with other SK features

**🚀 Next Steps:**
- Try with your own OpenAPI specifications
- Experiment with authenticated APIs
- Combine multiple OpenAPI tools in one agent
- Explore rate limiting and error handling

**Happy API integrating!** 🎉